In [ ]:
!pip install gdown

In [13]:
# ID de la carpeta de Google Drive que contiene todos los archivos
folder_id = "18EunmjOJsbE5Lh9zA0cZ4wKV6Um46dkg"
!gdown --folder {folder_id}

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY]
             [--speed SPEED] [--no-cookies] [--no-check-certificate]
             [--continue] [--folder] [--remaining-ok] [--format FORMAT]
             [--user-agent USER_AGENT]
             url_or_id
gdown: error: unrecognized arguments: --cookies


In [ ]:
import os

folder_name = "CompCars"
os.chdir(folder_name)

PASSWORD = "d89551fd190e38"

# 1. Unir todos los archivos en un único ZIP llamado "combined.zip"
!zip -F data.zip --out combined.zip

# 2. Descomprimir el archivo combinado usando la contraseña
output_directory = "/content/dataset_final"
os.makedirs(output_directory, exist_ok=True)

!unzip -P {PASSWORD} combined.zip -d {output_directory}

os.chdir('/content/')

FileNotFoundError: [Errno 2] No such file or directory: 'CompCars'

In [ ]:
!rm -r "/content/CompCars"

In [ ]:
# Comprobar el tamaño de las imágenes
from PIL import Image

im = Image.open(r"/content/dataset_final/data/image/1/1101/2011/2272c7d324cf79.jpg")

im.size  ## Es fácil ver que las imágenes tienen diferentes tamaños


(908, 626)

In [ ]:
import tensorflow as tf

# Definimos el tamaño de imagen objetivo para ResNet50
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

dataset_dir = "/content/dataset_final/data/image/"

# Crear el dataset de entrenamiento
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',          # Infiere las etiquetas desde los nombres de las carpetas
    label_mode='categorical',   # Convierte las etiquetas a formato one-hot
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,       # Usa el 20% de los datos para validación
    subset='training',          # Especifica que este es el conjunto de entrenamiento
    seed=123                    # Semilla para que la división sea reproducible
)

# Crear el dataset de validación
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=123
)



NotFoundError: Could not find directory /content/dataset_final/data/image/

In [ ]:
import tensorflow as tf

base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# --- Construir la cabeza y el modelo final ---

# Número de clases de tu dataset
NUM_CLASSES = 163

# Congelamos la base para no alterar sus pesos en el primer entrenamiento
base_model.trainable = False

# Construimos el modelo
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False) # Pasamos la entrada a la base
x = tf.keras.layers.GlobalAveragePooling2D()(x) # Aplicamos pooling
x = tf.keras.layers.Dropout(0.3)(x) # Aplicamos dropout para regularización
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x) # Capa de salida final

# Creamos el modelo completo
model = tf.keras.Model(inputs, outputs)

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 163)            │       333,987 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,921,699 (91.25 MB)

 Trainable params: 333,987 (1.27 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# --- Compilar el modelo ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# --- Entrenar solo la cabeza (útima capa) ---
NUM_EPOCHS_HEAD = 10

print("Entrenando la cabeza del modelo...")
history = model.fit(
    train_dataset,
    epochs=NUM_EPOCHS_HEAD,
    validation_data=validation_dataset
)

Entrenando la cabeza del modelo...
Epoch 1/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 564s 161ms/step - accuracy: 0.1212 - loss: 4.1302 - val_accuracy: 0.2513 - val_loss: 3.1638
Epoch 2/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 531s 154ms/step - accuracy: 0.2337 - loss: 3.2721 - val_accuracy: 0.2861 - val_loss: 2.9719
Epoch 3/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 477s 139ms/step - accuracy: 0.2618 - loss: 3.1364 - val_accuracy: 0.2876 - val_loss: 3.0094
Epoch 4/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 474s 139ms/step - accuracy: 0.2729 - loss: 3.0807 - val_accuracy: 0.3114 - val_loss: 2.8984
Epoch 5/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 479s 140ms/step - accuracy: 0.2805 - loss: 3.0490 - val_accuracy: 0.3278 - val_loss: 2.8241
Epoch 6/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 515s 144ms/step - accuracy: 0.2882 - loss: 3.0295 - val_accuracy: 0.3153 - val_loss: 2.8808
Epoch 7/10
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 494s 144ms/step - accuracy: 0.2916 - loss: 3.0126 - val_accuracy: 0.3302 - val_loss: 2.8305
Epoch 8/10
3419/3419 ━━━━━━

In [ ]:
# Guardar el modelo completo en el formato .keras
model_filename = "clasificador_coches_v1.keras"
model.save(model_filename)


Modelo guardado exitosamente en: clasificador_coches_v1.keras


In [ ]:
# Cargamos el modelo guardado anteriormente
model_path = "/content/clasificador_coches_v1.keras"
model = tf.keras.models.load_model(model_path)


ValueError: File not found: filepath=/content/clasificador_coches_v1.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
# --- Descongelar y afinar ---

# Descongelamos la base
base_model = model.get_layer('resnet50')
base_model.trainable = True

# Volvemos a compilar el modelo con una tasa de aprendizaje MUY BAJA
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # 0.00001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continuamos el entrenamiento
NUM_EPOCHS_FINE_TUNE = 10
total_epochs = NUM_EPOCHS_HEAD + NUM_EPOCHS_FINE_TUNE

print("\nIniciando el fine-tuning de todo el modelo...")
history_fine_tune = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=10,
    validation_data=validation_dataset
)


Iniciando el fine-tuning de todo el modelo...
Epoch 10/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1148s 318ms/step - accuracy: 0.2699 - loss: 3.5426 - val_accuracy: 0.5151 - val_loss: 1.9110
Epoch 11/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1148s 321ms/step - accuracy: 0.5109 - loss: 1.8473 - val_accuracy: 0.6113 - val_loss: 1.4923
Epoch 12/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1099s 321ms/step - accuracy: 0.6466 - loss: 1.2874 - val_accuracy: 0.6694 - val_loss: 1.2630
Epoch 13/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1057s 309ms/step - accuracy: 0.7516 - loss: 0.8863 - val_accuracy: 0.7024 - val_loss: 1.1375
Epoch 14/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1143s 321ms/step - accuracy: 0.8295 - loss: 0.5993 - val_accuracy: 0.7254 - val_loss: 1.0479
Epoch 15/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1098s 321ms/step - accuracy: 0.8849 - loss: 0.3979 - val_accuracy: 0.7429 - val_loss: 1.0054
Epoch 16/20
3419/3419 ━━━━━━━━━━━━━━━━━━━━ 1058s 308ms/step - accuracy: 0.9280 - loss: 0.2578 - val_accuracy: 0.7486 - val_loss: 0.9873
E

In [ ]:
# Guardamos la siguiente versión del modelo
model_filename = "clasificador_coches_v2.keras"
model.save(model_filename)